<a href="https://colab.research.google.com/github/tsm-mehmetakiftasoz/tsm_makif/blob/main/%D0%93%D0%BE%D1%82%D0%BE%D0%B2%D0%BE_%D0%92%D1%81%D0%B5%20%D0%B2%20%D0%BE%D0%B4%D0%BD%D0%BE%D0%BC_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Предназначено для объединения всех отчетов. Имена должны быть строго идентичными, и при выгрузке из SAP они должны быть в формате MAKIF_REPORT, иначе код не будет работать!!!

Необходимые отчеты:
*   ME5A_2022, ME5A_2023, ME5A_2024, ME5A_2025 (могут быть выгружены из SAP в формате 01.01.202X-31.12.202X)
*   ME2N_2022, ME2N_2023, ME2N_2024, ME2N_2025 (могут быть выгружены из SAP в формате 01.01.202X-31.12.202X)
*   zmm059_2022, zmm059_2023, zmm059_2025, zmm059_2024_1, zmm059_2024_2, zmm059_2024_3, zmm059_2024_4.(Отчеты за 2022, 2023, 2025 годы могут быть выгружены из SAP в формате 01.01.202X-31.12.202X. Для 2024 года граничные даты: 23.04.2024, 15.07.2024, 29.11.2024 и 01.12.2024 по 31.12.2024 (на 30.11 система выдает ошибку)).
*   zmb51_2022, zmb51_2023, zmb51_2025_1 (01.01.25-31.05.25), zmb51_2025_2 (01.06.2025-31.12.2025), zmb51_2024_1 (01.01-30.04.24), zmb51_2024_2 (01.05.2024-31.07.2024), zmb51_2024_3 (01.08.2024-30.09.2024), zmb51_2024_4 (01.10.2024-30.11.2024), zmb51_2024_5 (01.12.2024-31.12.2024)
*   ZMM067_2022(01.01.202X-31.12.202X) , ZMM067_2023(01.01.202X-31.12.202X), ZMM067_2024_1(01.01-31.05.24),ZMM067_2024_2(01.06-31.12.24),ZMM067_2025(01.01.202X-31.12.202X).

In [ ]:
!pip install pandas
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 12.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import os
import zipfile

In [ ]:
def duzenle_sayi_sutunu(df, sutun_adi):
    """
    Sayısal değerleri metin olarak tutan ve binlik/ondalık ayracı içeren sütunu dönüştürür.
    - Nokta (.) binlik ayracı olarak kaldırılır
    - Virgül (,) ondalık ayracı olarak noktaya çevrilir
    - Float olarak dönüştürülür
    - Eğer tam sayı ise integer olarak, değilse float olarak kalır
    - Sonuç string olarak döndürülür (görsel temizlik için)

    Parametreler:
    df : pd.DataFrame
    sutun_adi : str

    Dönüş:
    df (işlenmiş hali)
    """
    df[sutun_adi] = (
        df[sutun_adi]
        .astype(str)
        .str.replace(" ", "", regex=False)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
        .astype(float)
        .apply(lambda x: str(int(x)) if x.is_integer() else str(x))
    )
    return df


In [ ]:
def temizle_ve_cevir(x):
    try:
        return str(int(float(str(x).strip())))
    except:
        return str(x).strip()


In [ ]:
# Temizlenecek değerler listesi
temizlenecekler = ["nan", "Nan", "NaN", ""," ","0","naN","nAn","NAn","nAN"]

def temizle_nan(value):
    if str(value).strip().lower() in [x.lower() for x in temizlenecekler]:
        return pd.NA
    return value


In [ ]:
# Güvenli filtreleme fonksiyonu
def dolu_mi(x):
    return pd.notna(x) and str(x).strip() != "" and str(x).strip().upper() != "<NA>"


In [ ]:
#me5a'nın yüklenmesi
df_me5a_2022 = pd.read_csv('/content/ME5A_2022.csv', sep=None, engine='python')
df_me5a_2023 = pd.read_csv('/content/ME5A_2023.csv', sep=None, engine='python')
df_me5a_2024 = pd.read_csv('/content/ME5A_2024.csv', sep=None, engine='python')
df_me5a_2025 = pd.read_csv('/content/ME5A_2025.csv', sep=None, engine='python')
# sep=None will automatically detect the delimiter
# engine='python' is slower but more robust for complex cases
#zmm059un yüklenmesi
df_zmm059_2022 = pd.read_csv('/content/zmm059_2022.csv', sep=None, engine='python')
df_zmm059_2023 = pd.read_csv('/content/zmm059_2023.csv', sep=None, engine='python')
df_zmm059_2024_1 = pd.read_csv('/content/zmm059_2024_1.csv', sep=None, engine='python')
df_zmm059_2024_2 = pd.read_csv('/content/zmm059_2024_2.csv', sep=None, engine='python')
df_zmm059_2024_3 = pd.read_csv('/content/zmm059_2024_3.csv', sep=None, engine='python')
df_zmm059_2024_4 = pd.read_csv('/content/zmm059_2024_4.csv', sep=None, engine='python')
df_zmm059_2025 = pd.read_csv('/content/zmm059_2025.csv', sep=None, engine='python')
#yardımcı tabloların yüklenmesi
df_tedarikci_listesi = pd.read_csv('/content/tedarikci_isim.csv', sep=None, engine='python')
df_material_list = pd.read_csv('/content/material_list.csv', sep=None, engine='python')
# sep=None will automatically detect the delimiter
# engine='python' is slower but more robust for complex cases
#me2n'in yüklenmesi
df_me2n_2022 = pd.read_csv('/content/ME2N_2022.csv', sep=None, engine='python')
df_me2n_2023 = pd.read_csv('/content/ME2N_2023.csv', sep=None, engine='python')
df_me2n_2024 = pd.read_csv('/content/ME2N_2024.csv', sep=None, engine='python')
df_me2n_2025 = pd.read_csv('/content/ME2N_2025.csv', sep=None, engine='python')
# sep=None will automatically detect the delimiter
# engine='python' is slower but more robust for complex cases
#zmb51'in yüklenmesi
df_zmb51_2022 = pd.read_csv('/content/zmb51_2022.csv', sep=None, engine='python')
df_zmb51_2023 = pd.read_csv('/content/zmb51_2023.csv', sep=None, engine='python')
df_zmb51_2024_1 = pd.read_csv('/content/zmb51_2024_1.csv', sep=None, engine='python')
df_zmb51_2024_2 = pd.read_csv('/content/zmb51_2024_2.csv', sep=None, engine='python')
df_zmb51_2024_3 = pd.read_csv('/content/zmb51_2024_3.csv', sep=None, engine='python')
df_zmb51_2024_4 = pd.read_csv('/content/zmb51_2024_4.csv', sep=None, engine='python')
df_zmb51_2024_5 = pd.read_csv('/content/zmb51_2024_5.csv', sep=None, engine='python')
#df_zmb51_2024_6 = pd.read_csv('/content/zmb51_2024_6.csv', sep=None, engine='python')
#df_zmb51_2024_7 = pd.read_csv('/content/zmb51_2024_7.csv', sep=None, engine='python')
df_zmb51_2025 = pd.read_csv('/content/zmb51_2025.csv', sep=None, engine='python')
df_zmb51_2025_2 = pd.read_csv('/content/zmb51_2025-2.csv', sep=None, engine='python')
# sep=None will automatically detect the delimiter
# engine='python' is slower but more robust for complex cases
#zmm067'in yüklenmesi
df_zmm067_2022 = pd.read_csv('/content/ZMM067_2022.csv', sep=None, engine='python')
df_zmm067_2023 = pd.read_csv('/content/ZMM067_2023.csv', sep=None, engine='python')
df_zmm067_2024_1 = pd.read_csv('/content/ZMM067_2024_1.csv', sep=None, engine='python')
df_zmm067_2024_2 = pd.read_csv('/content/ZMM067_2024_2.csv', sep=None, engine='python')
df_zmm067_2025 = pd.read_csv('/content/ZMM067_2025.csv', sep=None, engine='python')



In [ ]:
#ilk sutuna yıl yazdırma me5a
df_me5a_2022["ГОД"] = "2022"
cols = ["ГОД"] + [col for col in df_me5a_2022.columns if col != "ГОД"]
df_me5a_2022 = df_me5a_2022[cols]
#
df_me5a_2023["ГОД"] = "2023"
cols = ["ГОД"] + [col for col in df_me5a_2023.columns if col != "ГОД"]
df_me5a_2023 = df_me5a_2023[cols]
#
df_me5a_2024["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_me5a_2024.columns if col != "ГОД"]
df_me5a_2024 = df_me5a_2024[cols]
#
df_me5a_2025["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_me5a_2025.columns if col != "ГОД"]
df_me5a_2025 = df_me5a_2025[cols]

#ilk sutuna yıl yazdırma zmm059
df_zmm059_2022["ГОД"] = "2022"
cols = ["ГОД"] + [col for col in df_zmm059_2022.columns if col != "ГОД"]
df_zmm059_2022 = df_zmm059_2022[cols]
#
df_zmm059_2023["ГОД"] = "2023"
cols = ["ГОД"] + [col for col in df_zmm059_2023.columns if col != "ГОД"]
df_zmm059_2023 = df_zmm059_2023[cols]
#
df_zmm059_2025["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_zmm059_2025.columns if col != "ГОД"]
df_zmm059_2025 = df_zmm059_2025[cols]
#
df_zmm059_2024_1["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm059_2024_1.columns if col != "ГОД"]
df_zmm059_2024_1 = df_zmm059_2024_1[cols]
#
df_zmm059_2024_2["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm059_2024_2.columns if col != "ГОД"]
df_zmm059_2024_2 = df_zmm059_2024_2[cols]
#
df_zmm059_2024_3["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm059_2024_3.columns if col != "ГОД"]
df_zmm059_2024_3 = df_zmm059_2024_3[cols]
#
df_zmm059_2024_4["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm059_2024_4.columns if col != "ГОД"]
df_zmm059_2024_4 = df_zmm059_2024_4[cols]

#ilk sutuna yıl yazdırma me2n
df_me2n_2022["ГОД"] = "2022"
cols = ["ГОД"] + [col for col in df_me2n_2022.columns if col != "ГОД"]
df_me2n_2022 = df_me2n_2022[cols]
#
df_me2n_2023["ГОД"] = "2023"
cols = ["ГОД"] + [col for col in df_me2n_2023.columns if col != "ГОД"]
df_me2n_2023 = df_me2n_2023[cols]
#
df_me2n_2024["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_me2n_2024.columns if col != "ГОД"]
df_me2n_2024 = df_me2n_2024[cols]
#
df_me2n_2025["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_me2n_2025.columns if col != "ГОД"]
df_me2n_2025 = df_me2n_2025[cols]
#ilk sutuna yıl yazdırma zmb51
df_zmb51_2022["ГОД"] = "2022"
cols = ["ГОД"] + [col for col in df_zmb51_2022.columns if col != "ГОД"]
df_zmb51_2022 = df_zmb51_2022[cols]
#
df_zmb51_2023["ГОД"] = "2023"
cols = ["ГОД"] + [col for col in df_zmb51_2023.columns if col != "ГОД"]
df_zmb51_2023 = df_zmb51_2023[cols]
#
df_zmb51_2024_1["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmb51_2024_1.columns if col != "ГОД"]
df_zmb51_2024_1 = df_zmb51_2024_1[cols]
#
df_zmb51_2024_2["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmb51_2024_2.columns if col != "ГОД"]
df_zmb51_2024_2 = df_zmb51_2024_2[cols]
#
df_zmb51_2024_3["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmb51_2024_3.columns if col != "ГОД"]
df_zmb51_2024_3 = df_zmb51_2024_3[cols]
#
df_zmb51_2024_4["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmb51_2024_4.columns if col != "ГОД"]
df_zmb51_2024_4 = df_zmb51_2024_4[cols]
#
df_zmb51_2024_5["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmb51_2024_5.columns if col != "ГОД"]
df_zmb51_2024_5 = df_zmb51_2024_5[cols]
#
df_zmb51_2025["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_zmb51_2025.columns if col != "ГОД"]
df_zmb51_2025 = df_zmb51_2025[cols]
#
df_zmb51_2025_2["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_zmb51_2025_2.columns if col != "ГОД"]
df_zmb51_2025_2 = df_zmb51_2025_2[cols]
#ilk sutuna yıl yazdırma zmm067
# 2022
df_zmm067_2022["ГОД"] = "2022"
cols = ["ГОД"] + [col for col in df_zmm067_2022.columns if col != "ГОД"]
df_zmm067_2022 = df_zmm067_2022[cols]

# 2023
df_zmm067_2023["ГОД"] = "2023"
cols = ["ГОД"] + [col for col in df_zmm067_2023.columns if col != "ГОД"]
df_zmm067_2023 = df_zmm067_2023[cols]

# 2024_1
df_zmm067_2024_1["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm067_2024_1.columns if col != "ГОД"]
df_zmm067_2024_1 = df_zmm067_2024_1[cols]

# 2024_2
df_zmm067_2024_2["ГОД"] = "2024"
cols = ["ГОД"] + [col for col in df_zmm067_2024_2.columns if col != "ГОД"]
df_zmm067_2024_2 = df_zmm067_2024_2[cols]

# 2025
df_zmm067_2025["ГОД"] = "2025"
cols = ["ГОД"] + [col for col in df_zmm067_2025.columns if col != "ГОД"]
df_zmm067_2025 = df_zmm067_2025[cols]

In [ ]:
#me5a birleştirme
df_listesi_me5a = [df_me5a_2025,df_me5a_2024, df_me5a_2023,df_me5a_2022 ]
df_me5a_2022_2025 = pd.concat(df_listesi_me5a, ignore_index=True)
# zmm059 birleştirme
df_listesi_zmm059 = [df_zmm059_2025, df_zmm059_2024_4, df_zmm059_2024_3, df_zmm059_2024_2, df_zmm059_2024_1, df_zmm059_2023,df_zmm059_2022 ]
df_zmm059_2022_2025 = pd.concat(df_listesi_zmm059, ignore_index=True)
#me2n birleştirme
df_listesi_me2n = [df_me2n_2025, df_me2n_2024, df_me2n_2023,df_me2n_2022]
df_me2n_2022_2025 = pd.concat(df_listesi_me2n, ignore_index=True)
#zmb51 birleştirme
df_listesi_zmb51 = [df_zmb51_2025_2,df_zmb51_2025,df_zmb51_2024_5,df_zmb51_2024_4,df_zmb51_2024_3,
                    df_zmb51_2024_2,df_zmb51_2024_1,df_zmb51_2023,df_zmb51_2022
    ]
birlesik_df_zmb51 = pd.concat(df_listesi_zmb51, ignore_index=True)
#zmm067 birleştirme
df_listesi_zmm067 = [df_zmm067_2025,df_zmm067_2024_2,df_zmm067_2024_1,df_zmm067_2023,df_zmm067_2022]
df_zmm067_2022_2025 = pd.concat(df_listesi_zmm067, ignore_index=True)

In [ ]:
#yedek tablolar
df_me5a_2022_2025_yedek=df_me5a_2022_2025.copy()
df_zmm059_2022_2025_yedek=df_zmm059_2022_2025.copy()
df_me2n_2022_2025_yedek=df_me2n_2022_2025.copy()
birlesik_df_zmb51_yedek=birlesik_df_zmb51.copy()
df_zmm067_2022_2025_yedek=df_zmm067_2022_2025.copy()

In [ ]:
#zmm059
df_zmm059_2022_2025["﻿Документ закупки"] = df_zmm059_2022_2025["﻿Документ закупки"].apply(temizle_ve_cevir)
df_zmm059_2022_2025.rename(columns={'﻿Документ закупки': 'Документ_закупки'}, inplace=True)
df_zmm059_2022_2025["Материал"] = df_zmm059_2022_2025["Материал"].apply(temizle_ve_cevir)
df_zmm059_2022_2025["Заявка"] = df_zmm059_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Количество ЗП")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Минимальная цена ЗП")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Единица ЗП")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Цена нетто ЗП")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Итого цена нетто")
df_zmm059_2022_2025["Дата создания"] = pd.to_datetime(df_zmm059_2022_2025["Дата создания"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025["Дата поставки"] = pd.to_datetime(df_zmm059_2022_2025["Дата поставки"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025["Дата доставки, одобренная Поставщиком"] = pd.to_datetime(df_zmm059_2022_2025["Дата доставки, одобренная Поставщиком"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025["Группа закупок"] = df_zmm059_2022_2025["Группа закупок"].apply(temizle_ve_cevir)
df_zmm059_2022_2025["БЕ"] = df_zmm059_2022_2025["БЕ"].apply(temizle_ve_cevir)
df_zmm059_2022_2025["Закуп. организация"] = df_zmm059_2022_2025["Закуп. организация"].apply(temizle_ve_cevir)
df_zmm059_2022_2025["Поставщик"] = df_zmm059_2022_2025["Поставщик"].apply(temizle_ve_cevir)
df_zmm059_2022_2025["Дата создания.1"] = pd.to_datetime(df_zmm059_2022_2025["Дата создания.1"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Цена заказа нетто")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Стоимость брутто")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Недопоставленное количество")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Количество поставки")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Сумма брутто")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Количество по счету")
df_zmm059_2022_2025["Дата проводки"] = pd.to_datetime(df_zmm059_2022_2025["Дата проводки"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Количество по счету.1")
df_zmm059_2022_2025 = duzenle_sayi_sutunu(df_zmm059_2022_2025, "Остаток количества по счету")
df_zmm059_2022_2025["Дата контракта"] = pd.to_datetime(df_zmm059_2022_2025["Дата контракта"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025["Дата вступления в силу контракта"] = pd.to_datetime(df_zmm059_2022_2025["Дата вступления в силу контракта"], format="%d.%m.%Y", errors="coerce").dt.date
df_zmm059_2022_2025["Дата окончания действия договора"] = pd.to_datetime(df_zmm059_2022_2025["Дата окончания действия договора"], format="%d.%m.%Y", errors="coerce").dt.date


In [ ]:
df_me5a_2022_2025.columns

Index(['ГОД', '﻿Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования'],
      dtype='object')

In [ ]:
#ME5A
df_me5a_2022_2025["﻿Заявка"] = df_me5a_2022_2025["﻿Заявка"].apply(temizle_ve_cevir)
df_me5a_2022_2025.rename(columns={'﻿Заявка': 'Заявка'}, inplace=True)
df_me5a_2022_2025.rename(columns={'Индикатор удаления': 'Индикатор удаления_B3'}, inplace=True)
df_me5a_2022_2025["Материал"] = df_me5a_2022_2025["Материал"].apply(temizle_ve_cevir)
df_me5a_2022_2025["'Дата заявки"] = pd.to_datetime(df_me5a_2022_2025["Дата заявки"], format="%d.%m.%Y", errors="coerce").dt.date
df_me5a_2022_2025["Группа закупок"] = df_me5a_2022_2025["Группа закупок"].apply(temizle_ve_cevir)
df_me5a_2022_2025["'Дата заявки"] = pd.to_datetime(df_me5a_2022_2025["Дата заявки"], format="%d.%m.%Y", errors="coerce").dt.date
df_me5a_2022_2025["Дата изменения"] = df_me5a_2022_2025["Дата изменения"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Заказ на поставку"] = df_me5a_2022_2025["Заказ на поставку"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Позиция ЗкзНаПостав"] = df_me5a_2022_2025["Позиция ЗкзНаПостав"].apply(temizle_ve_cevir)
df_me5a_2022_2025 = duzenle_sayi_sutunu(df_me5a_2022_2025, "ЗатребованКолич")
df_me5a_2022_2025 = duzenle_sayi_sutunu(df_me5a_2022_2025, "Заказанное к-во")
df_me5a_2022_2025["Закуп. организация"] = df_me5a_2022_2025["Закуп. организация"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Номер проекта"] = df_me5a_2022_2025["Номер проекта"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Дата деблокирования"] = pd.to_datetime(df_me5a_2022_2025["Дата деблокирования"], format="%d.%m.%Y", errors="coerce").dt.date


In [ ]:
df_me5a_cleaned =df_me5a_2022_2025.copy()

In [ ]:
#me2n
df_me2n_2022_2025["﻿Документ закупки"] = df_me2n_2022_2025["﻿Документ закупки"].apply(temizle_ve_cevir)
df_me2n_2022_2025.rename(columns={'﻿Документ закупки': 'Документ закупки'}, inplace=True)
df_me2n_2022_2025["Дата документа"] = pd.to_datetime(df_me2n_2022_2025["Дата документа"], format="%d.%m.%Y", errors="coerce").dt.date
df_me2n_2022_2025["Заявка"] = df_me2n_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_me2n_2022_2025["Материал"] = df_me2n_2022_2025["Материал"].apply(temizle_ve_cevir)
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "Объем заказа")
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "еще поставить (количество)")
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "Количество в СЕИ")
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "Цена нетто")
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "СтоимЗаказа нетто")
df_me2n_2022_2025 = duzenle_sayi_sutunu(df_me2n_2022_2025, "Еще для поставки (стоимость)")
df_me2n_2022_2025["Дата поставки"] = pd.to_datetime(df_me2n_2022_2025["Дата поставки"], format="%d.%m.%Y", errors="coerce").dt.date


In [ ]:
#zmb51
birlesik_df_zmb51 = duzenle_sayi_sutunu(birlesik_df_zmb51, "Сумма во ВВ")
birlesik_df_zmb51["Партия"] = birlesik_df_zmb51["Партия"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Материал"] = birlesik_df_zmb51["Материал"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Дата ввода"] = pd.to_datetime(birlesik_df_zmb51["Дата ввода"], format="%d.%m.%Y", errors="coerce").dt.date
birlesik_df_zmb51["Дата документа"] = pd.to_datetime(birlesik_df_zmb51["Дата документа"], format="%d.%m.%Y", errors="coerce").dt.date
birlesik_df_zmb51["Дата проводки"] = pd.to_datetime(birlesik_df_zmb51["Дата проводки"], format="%d.%m.%Y", errors="coerce").dt.date
birlesik_df_zmb51["Внутренний заказ"] = birlesik_df_zmb51["Внутренний заказ"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Заказ на поставку"] = birlesik_df_zmb51["Заказ на поставку"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Позиция"] = birlesik_df_zmb51["Позиция"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Документ материала"] = birlesik_df_zmb51["Документ материала"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Поз. док. материала"] = birlesik_df_zmb51["Поз. док. материала"].apply(temizle_ve_cevir)
birlesik_df_zmb51 = duzenle_sayi_sutunu(birlesik_df_zmb51, "Кол-во в ЕИ ввода")
birlesik_df_zmb51 = duzenle_sayi_sutunu(birlesik_df_zmb51, "Количество")
birlesik_df_zmb51['Поставщик'] = birlesik_df_zmb51['Поставщик'].apply(temizle_ve_cevir)
birlesik_df_zmb51.rename(columns={'﻿Вид движения': 'Вид движения'}, inplace=True)


In [ ]:
#zmm067
df_zmm067_2022_2025["﻿Заявка"] = df_zmm067_2022_2025["﻿Заявка"].apply(temizle_ve_cevir)
df_zmm067_2022_2025.rename(columns={'﻿Заявка': 'Заявка'}, inplace=True)
df_zmm067_2022_2025["Позиция заявки"] = df_zmm067_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)
df_zmm067_2022_2025["Материал"] = df_zmm067_2022_2025["Материал"].apply(temizle_ve_cevir)
df_zmm067_2022_2025["Заказ на поставку"] = df_zmm067_2022_2025["Заказ на поставку"].apply(temizle_ve_cevir)
df_zmm067_2022_2025["Позиция ЗкзНаПостав"] = df_zmm067_2022_2025["Позиция ЗкзНаПостав"].apply(temizle_ve_cevir)



In [ ]:
#df_zmm067_2022_2025["Позиция ЗкзНаПостав"] = df_zmm067_2022_2025["Позиция ЗкзНаПостав"].apply(temizle_ve_cevir)


In [ ]:
#df_zmm067_2022_2025["Дополнительное"].sample(n=5)

In [ ]:
#df_zmm067_2022_2025.columns

In [ ]:
df_me5a_2022_2025 = df_me5a_2022_2025.map(temizle_nan)
df_zmm059_2022_2025 = df_zmm059_2022_2025.map(temizle_nan)
df_me2n_2022_2025 = df_me2n_2022_2025.map(temizle_nan)
df_tedarikci_listesi = df_tedarikci_listesi.map(temizle_nan)
birlesik_df_zmb51 = birlesik_df_zmb51.map(temizle_nan)
df_zmm067_2022_2025 = df_zmm067_2022_2025.map(temizle_nan)

In [ ]:
df_calisilan = df_zmm059_2022_2025.copy()

In [ ]:
df_calisilan["B3-B3POZ"] = (
    df_calisilan["Заявка"].astype(str) + "_" + df_calisilan["Позиция заявки"].astype(str)
)
df_me5a_2022_2025["B3-B3POZ"] = (
    df_me5a_2022_2025["Заявка"].astype(str) + "_" + df_me5a_2022_2025["Позиция заявки"].astype(str)
)

In [ ]:
df_calisilan.columns.values[2] = "Зп_Поз"
df_tedarikci_listesi.rename(columns={'kod': 'Поставщик'}, inplace=True)

In [ ]:
adetler = df_calisilan["B3-B3POZ"].value_counts()

In [ ]:
mukerrer_kodlar = adetler[adetler > 1].index

In [ ]:
silinmesi_gerekenler = df_calisilan[
    (df_calisilan["B3-B3POZ"].isin(mukerrer_kodlar)) &
    (df_calisilan["Документ_закупки"].isna())

]

In [ ]:
df_calisilan = df_calisilan.drop(silinmesi_gerekenler.index)

In [ ]:
#df_calisilan["Поставщик"] = df_calisilan["Поставщик"].apply(temizle_ve_cevir)

In [ ]:
# Tedarikçi sözlüğünü yine string anahtarlara göre oluştur
tedarikci_dict = dict(zip(df_tedarikci_listesi.iloc[:, 0].astype(str), df_tedarikci_listesi.iloc[:, 1]))

# Eşleştirme
df_calisilan["имя_поставщика"] = df_calisilan["Поставщик"].map(tedarikci_dict)

In [ ]:
#df_calisilan.tail()

In [ ]:
# Sözlük oluştur (anahtar: B3-B3POZ, değer: Материал)
b3_to_material_dict = dict(zip(df_me5a_2022_2025["B3-B3POZ"].astype(str), df_me5a_2022_2025["Материал"]))

In [ ]:
# B3-B3POZ'e göre eşleşen Материал'ları getir (yeni bir sütun olarak)
df_calisilan["Материал_yeni"] = df_calisilan["B3-B3POZ"].astype(str).map(b3_to_material_dict)

In [ ]:
# Sadece boş olan Материал hücrelerini, eşleşen değerle doldur
df_calisilan["Материал"] = df_calisilan["Материал"].fillna(df_calisilan["Материал_yeni"])
# Geçici sütunu silelim
df_calisilan.drop(columns=["Материал_yeni"], inplace=True)

In [ ]:
# Sözlükleri oluştur
ru_dict = dict(zip(df_material_list.iloc[:, 0].astype(str), df_material_list.iloc[:, 1]))
tr_dict = dict(zip(df_material_list.iloc[:, 0].astype(str), df_material_list.iloc[:, 2]))

# Eşleştir
df_calisilan["RU Наименование"] = df_calisilan["Материал"].astype(str).map(ru_dict)
df_calisilan["TR Наименование"] = df_calisilan["Материал"].astype(str).map(tr_dict)


In [ ]:
#df_calisilan["Материал"].isna().sum()

In [ ]:
#df_calisilan[["Поставщик", "имя_поставщика"]].sample(n=5)

In [ ]:
df_calisilan.drop(columns=['Материалы тур.', 'Материалы анг.'], inplace=True)

In [ ]:
# Sütunları çıkart
col_ru = df_calisilan.pop('RU Наименование')
col_tr = df_calisilan.pop('TR Наименование')
col_b3 = df_calisilan.pop('B3-B3POZ')

# Sütunları istediğin yerlere ekle
df_calisilan.insert(1, 'B3-B3POZ', col_b3)
df_calisilan.insert(8, 'RU Наименование', col_ru)
df_calisilan.insert(9, 'TR Наименование', col_tr)


In [ ]:
 # Sözlük oluştur (anahtar: B3-B3POZ, değer: Материал)
b3_to_material_dict = dict(zip(df_me5a_2022_2025["B3-B3POZ"].astype(str), df_me5a_2022_2025["Материал"]))

In [ ]:
# B3-B3POZ'e göre eşleşen Материал'ları getir (yeni bir sütun olarak)
df_calisilan["Материал_yeni"] = df_calisilan["B3-B3POZ"].astype(str).map(b3_to_material_dict)

In [ ]:
# Sadece boş olan Материал hücrelerini, eşleşen değerle doldur
df_calisilan["Материал"] = df_calisilan["Материал"].fillna(df_calisilan["Материал_yeni"])
# Geçici sütunu silelim
df_calisilan.drop(columns=["Материал_yeni"], inplace=True)

In [ ]:
# Sözlükleri oluştur
ru_dict = dict(zip(df_material_list.iloc[:, 0].astype(str), df_material_list.iloc[:, 1]))
tr_dict = dict(zip(df_material_list.iloc[:, 0].astype(str), df_material_list.iloc[:, 2]))

# Eşleştir
df_calisilan["RU Наименование"] = df_calisilan["Материал"].astype(str).map(ru_dict)
df_calisilan["TR Наименование"] = df_calisilan["Материал"].astype(str).map(tr_dict)

In [ ]:
df_calisilan["Материал"].isna().sum()

np.int64(4993)

In [ ]:
df_calisilan = df_calisilan.map(temizle_nan)

In [ ]:
sorted_columns = sorted(df_calisilan.columns)
print(sorted_columns)

['B3-B3POZ', 'RU Наименование', 'TR Наименование', 'Автор Внутреннего заказа', 'БЕ', 'Валюта', 'Валюта ЗП', 'Валюта.1', 'Валюта.2', 'Вид докум. закупки', 'Вид докум. закупки.1', 'ГОД', 'Группа закупок', 'Группа закупок.1', 'Группа материалов', 'Дата вступления в силу контракта', 'Дата доставки, одобренная Поставщиком', 'Дата контракта', 'Дата окончания действия договора', 'Дата поставки', 'Дата проводки', 'Дата создания', 'Дата создания.1', 'Документ_закупки', 'ЕИ заказа на постав.', 'Единица ЗП', 'Единица цены', 'Завод', 'Закуп. организация', 'Заявка', 'Значок "поставка завершена"', 'Зп_Поз', 'Имя 1', 'Инвентарный номер в SAS', 'Индикатор удаления', 'Итого цена нетто', 'Количество ЗП', 'Количество по счету', 'Количество по счету.1', 'Количество по счету.2', 'Количество поставки', 'Краткий текст', 'Материал', 'Минимальная цена ЗП', 'Минимальная цена ЗП.1', 'Минимальная цена ЗП.2', 'Название ГрЗакупок', 'Название ГрЗакупок.1', 'Название ЗакупОрг', 'Название группы', 'Название фирмы', 'Н

In [ ]:
#df_calisilan[["Документ_закупки","Зп_Поз","Количество ЗП","Цена нетто ЗП","Итого цена нетто"]].sample(n=5, random_state=1)

In [ ]:
# Hepsini string yap ve boşlukları temizle
birlesik_df_zmb51["Заказ на поставку"] = birlesik_df_zmb51["Заказ на поставку"].astype(str).str.strip()
df_me2n_2022_2025["Документ закупки"] = df_me2n_2022_2025["Документ закупки"].astype(str).str.strip()


In [ ]:
# "Наш знак" için dict
dict_nash_znak = dict(zip(df_me2n_2022_2025["Документ закупки"], df_me2n_2022_2025["Наш знак"]))

# "Ваш код" için dict
dict_vash_kod = dict(zip(df_me2n_2022_2025["Документ закупки"], df_me2n_2022_2025["Ваш код"]))

# map ile ekleme
birlesik_df_zmb51["Наш знак"] = birlesik_df_zmb51["Заказ на поставку"].map(dict_nash_znak)
birlesik_df_zmb51["Ваш код"] = birlesik_df_zmb51["Заказ на поставку"].map(dict_vash_kod)

In [ ]:
df_calisilan["Позиция заявки"].apply(temizle_ve_cevir)

,Позиция заявки
0,2920
1,2930
2,2940
3,2910
4,2900
...,...
751753,<NA>
751754,<NA>
751755,<NA>
751756,<NA>


In [ ]:
# 1. Anahtar sütunlarını birleştir ve zp_zppoz adını ver
birlesik_df_zmb51["zp_zppoz"] = (
    birlesik_df_zmb51["Заказ на поставку"].apply(temizle_ve_cevir) + "_" +
    birlesik_df_zmb51["Позиция"].apply(temizle_ve_cevir)
)

df_calisilan["zp_zppoz"] = (
    df_calisilan["Документ_закупки"].apply(temizle_ve_cevir) + "_" +
    df_calisilan["Зп_Поз"].apply(temizle_ve_cevir)
)

# 2. dict oluştur
dict_poz = dict(zip(df_calisilan["zp_zppoz"], df_calisilan["Позиция заявки"]))

# 3. map ile yeni sütunu ekle
birlesik_df_zmb51["Позиция заявки"] = birlesik_df_zmb51["zp_zppoz"].map(dict_poz)
birlesik_df_zmb51["Позиция заявки"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Позиция заявки"] = birlesik_df_zmb51["Позиция заявки"].fillna(0).astype(int)

/tmp/ipython-input-873327287.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  birlesik_df_zmb51["Позиция заявки"] = birlesik_df_zmb51["Позиция заявки"].fillna(0).astype(int)


In [ ]:
# 1. Zorunlu temizleme işlemleri (orijinal sütunların üzerine yazar)
df_me5a_2022_2025["Заявка"] = df_me5a_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Позиция заявки"] = df_me5a_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)

df_zmm067_2022_2025["Заявка"] = df_zmm067_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_zmm067_2022_2025["Позиция заявки"] = df_zmm067_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
df_me5a_2022_2025["zayavka_poz"] = df_me5a_2022_2025["Заявка"] + "_" + df_me5a_2022_2025["Позиция заявки"]
df_zmm067_2022_2025["zayavka_poz"] = df_zmm067_2022_2025["Заявка"] + "_" + df_zmm067_2022_2025["Позиция заявки"]

# 3. Dict oluştur
dict_dop = dict(zip(df_zmm067_2022_2025["zayavka_poz"], df_zmm067_2022_2025["Дополнительное"]))

# 4. map ile yeni sütun ekle
df_me5a_2022_2025["Дополнительное"] = df_me5a_2022_2025["zayavka_poz"].map(dict_dop)

#yeni oluşturulan sutunlara ihtiyaç kalmadı

df_me5a_2022_2025 = df_me5a_2022_2025.drop(columns=["zayavka_poz"])
df_zmm067_2022_2025 = df_zmm067_2022_2025.drop(columns=["zayavka_poz"])

In [ ]:
# 1. Sütunları temizle (aynı sütunların üzerine yazıyoruz)
df_me2n_2022_2025["Документ закупки"] = df_me2n_2022_2025["Документ закупки"].apply(temizle_ve_cevir)
df_me2n_2022_2025["Позиция"] = df_me2n_2022_2025["Позиция"].apply(temizle_ve_cevir)

df_calisilan["Документ_закупки"] = df_calisilan["Документ_закупки"].apply(temizle_ve_cevir)
df_calisilan["Зп_Поз"] = df_calisilan["Зп_Поз"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
df_me2n_2022_2025["doc_poz"] = df_me2n_2022_2025["Документ закупки"] + "_" + df_me2n_2022_2025["Позиция"]
df_calisilan["doc_poz"] = df_calisilan["Документ_закупки"] + "_" + df_calisilan["Зп_Поз"]

# 3. Dict oluştur (anahtar: doc_poz, değer: Создал)
dict_sozdal = dict(zip(df_calisilan["doc_poz"], df_calisilan["Создал"]))

# 4. map ile df_me2n'e ekle
df_me2n_2022_2025["Создал"] = df_me2n_2022_2025["doc_poz"].map(dict_sozdal)

# yeni oluşturulan sütunlara ihtiyaç kalmadı
df_me2n_2022_2025 = df_me2n_2022_2025.drop(columns=["doc_poz"])
df_calisilan = df_calisilan.drop(columns=["doc_poz"])

In [ ]:
# 1. Temizleme işlemi
birlesik_df_zmb51["Поставщик"] = birlesik_df_zmb51["Поставщик"].apply(temizle_ve_cevir)
df_tedarikci_listesi["﻿Кредитор"] = df_tedarikci_listesi["﻿Кредитор"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur (tek sütun olduğu için direkt kullanabiliriz)
# 3. Dict oluştur
dict_suppliers = dict(zip(df_tedarikci_listesi["﻿Кредитор"], df_tedarikci_listesi["Имя поставщика"]))

# 4. map ile birlesik_df_zmb51'e ekle
birlesik_df_zmb51["Имя поставщика"] = birlesik_df_zmb51["Поставщик"].map(dict_suppliers)


In [ ]:
# 1. Temizleme işlemi
birlesik_df_zmb51["Заказ на поставку"] = birlesik_df_zmb51["Заказ на поставку"].apply(temizle_ve_cevir)
birlesik_df_zmb51["Позиция"] = birlesik_df_zmb51["Позиция"].apply(temizle_ve_cevir)

df_me2n_2022_2025["Документ закупки"] = df_me2n_2022_2025["Документ закупки"].apply(temizle_ve_cevir)
df_me2n_2022_2025["Позиция"] = df_me2n_2022_2025["Позиция"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
birlesik_df_zmb51["doc_poz"] = birlesik_df_zmb51["Заказ на поставку"] + "_" + birlesik_df_zmb51["Позиция"]
df_me2n_2022_2025["doc_poz"] = df_me2n_2022_2025["Документ закупки"] + "_" + df_me2n_2022_2025["Позиция"]

# 3. Dict oluştur (anahtar: doc_poz, değer: Объем заказа)
dict_obem = dict(zip(df_me2n_2022_2025["doc_poz"], df_me2n_2022_2025["Объем заказа"]))

# 4. map ile birlesik_df_zmb51'e ekle
birlesik_df_zmb51["Объем заказа"] = birlesik_df_zmb51["doc_poz"].map(dict_obem)
#yeni oluşturulan sutunlara ihtiyaç kalmadı
birlesik_df_zmb51 = birlesik_df_zmb51.drop(columns=["doc_poz"])
df_me2n_2022_2025 = df_me2n_2022_2025.drop(columns=["doc_poz"])



In [ ]:
df_me2n_2022_2025["Объем заказа"] = pd.to_numeric(df_me2n_2022_2025["Объем заказа"], errors="coerce")
df_me2n_2022_2025["еще поставить (количество)"] = pd.to_numeric(
    df_me2n_2022_2025["еще поставить (количество)"].fillna(0), errors="coerce"
)

# 2. Hesaplama
df_me2n_2022_2025["Уже поставлено (количество)"] = (
    df_me2n_2022_2025["Объем заказа"] - df_me2n_2022_2025["еще поставить (количество)"]
)

In [ ]:
# 1. Temizleme işlemi
df_me2n_2022_2025["Документ закупки"] = df_me2n_2022_2025["Документ закупки"].apply(temizle_ve_cevir)
df_me2n_2022_2025["Позиция"] = df_me2n_2022_2025["Позиция"].apply(temizle_ve_cevir)


df_calisilan["Документ_закупки"] = df_calisilan["Документ_закупки"].apply(temizle_ve_cevir)
df_calisilan["Зп_Поз"] = df_calisilan["Зп_Поз"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
df_me2n_2022_2025["doc_poz"] = df_me2n_2022_2025["Документ закупки"] + "_" + df_me2n_2022_2025["Позиция"]
df_calisilan["doc_poz"] = df_calisilan["Документ_закупки"] + "_" + df_calisilan["Зп_Поз"]

# 3. Dict oluştur (anahtar: doc_poz, değer: Позиция заявки)
dict_poz = dict(zip(df_calisilan["doc_poz"], df_calisilan["Позиция заявки"]))

# 4. map ile df_me2n_2022_2025'e ekle
df_me2n_2022_2025["Позиция заявки"] = df_me2n_2022_2025["doc_poz"].map(dict_poz)
#yeni oluşturulan sutunlara ihtiyaç kalmadı
df_calisilan = df_calisilan.drop(columns=["doc_poz"])
df_me2n_2022_2025 = df_me2n_2022_2025.drop(columns=["doc_poz"])


In [ ]:
df_me2n_2022_2025["Материал_ME2N"] = df_me2n_2022_2025["Материал"]
df_me5a_2022_2025["Материал_ME5A"] = df_me5a_2022_2025["Материал"]
# 1. Temizleme işlemi
df_me2n_2022_2025["Заявка"] = df_me2n_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_me2n_2022_2025["Позиция заявки"] = df_me2n_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)

df_me5a_2022_2025["Заявка"] = df_me5a_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_me5a_2022_2025["Позиция заявки"] = df_me5a_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
df_me2n_2022_2025["zayavka_poz"] = df_me2n_2022_2025["Заявка"] + "_" + df_me2n_2022_2025["Позиция заявки"]
df_me5a_2022_2025["zayavka_poz"] = df_me5a_2022_2025["Заявка"] + "_" + df_me5a_2022_2025["Позиция заявки"]

# 3. Dict oluştur (anahtar: zayavka_poz, değer: Материал_ME5A)
dict_me5a = dict(zip(df_me5a_2022_2025["zayavka_poz"], df_me5a_2022_2025["Материал_ME5A"]))

# 4. map ile df_me2n_2022_2025'e ekle
df_me2n_2022_2025["Материал_ME5A"] = df_me2n_2022_2025["zayavka_poz"].map(dict_me5a)
#yeni oluşturulan sutunlara ihtiyaç kalmadı
df_me5a_2022_2025 = df_me5a_2022_2025.drop(columns=["zayavka_poz"])
df_me2n_2022_2025 = df_me2n_2022_2025.drop(columns=["zayavka_poz"])


In [ ]:
df_me2n_2022_2025 = df_me2n_2022_2025.applymap(temizle_nan)

/tmp/ipython-input-2153232079.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_me2n_2022_2025 = df_me2n_2022_2025.applymap(temizle_nan)


In [ ]:
df_me2n_2022_2025_analog = df_me2n_2022_2025[
    (df_me2n_2022_2025["Материал_ME2N"] != df_me2n_2022_2025["Материал_ME5A"]) &
    (df_me2n_2022_2025["Заявка"].apply(dolu_mi) & df_me2n_2022_2025["Позиция заявки"].apply(dolu_mi))
].copy()


In [ ]:
# 1. Temizleme işlemi (gerekiyorsa)
df_me2n_2022_2025_analog["Материал_ME5A"] = df_me2n_2022_2025_analog["Материал_ME5A"].apply(temizle_ve_cevir)
df_material_list["﻿Материал"] = df_material_list["﻿Материал"].apply(temizle_ve_cevir)

# 2. Dict oluştur
dict_tr = dict(zip(df_material_list["﻿Материал"], df_material_list["TR Наименование"]))
dict_ru = dict(zip(df_material_list["﻿Материал"], df_material_list["RU Наименование"]))

# 3. map ile yeni sütunları ekle
df_me2n_2022_2025_analog["TR Наименование_ВЗ"] = df_me2n_2022_2025_analog["Материал_ME5A"].map(dict_tr)
df_me2n_2022_2025_analog["RU Наименование_ВЗ"] = df_me2n_2022_2025_analog["Материал_ME5A"].map(dict_ru)


In [ ]:
df_me2n_2022_2025_analog = df_me2n_2022_2025_analog.rename(columns={"Наименование RU": "Наименование RU_ЗП"})
df_me2n_2022_2025_analog = df_me2n_2022_2025_analog.rename(columns={"Наименование TR": "Наименование TR_ЗП"})


In [ ]:
# 1. Anahtar sütunları tuple haline getirelim
df_me2n_2022_2025["key"] = df_me2n_2022_2025[["Документ закупки", "Позиция"]].apply(tuple, axis=1)
df_me5a_2022_2025["key"] = df_me5a_2022_2025[["Заказ на поставку", "Позиция ЗкзНаПостав"]].apply(tuple, axis=1)

# 2. Aktarılacak sütunlar ve hedef adları
map_dict = {
    "Объем заказа": "Объем заказа",
    "еще поставить (количество)": "еще поставить (количество)",
    "Еще для поставки (стоимость)": "Еще для поставки (стоимость)",
    "Для фактурирования (колич.)": "Для фактурирования (колич.)",
    "Для фактурирования (стоим.)": "Для фактурирования (стоим.)",
    "Имя поставщика": "Имя поставщика",
    "Создал": "ЗП_Создал",
    "Наш знак":"Наш знак",
    "Цена нетто":"Цена нетто",
    "Валюта":"Валюта",
    "Индикатор удаления":"Индикатор удаления_зп",
    "ЕИ заказа на постав.":"ЕИ заказа на постав.",
    "Единица цены":"Единица цены",

}

# 3. Her sütun için dict map uygulama
for source_col, target_col in map_dict.items():
    temp_dict = dict(zip(df_me2n_2022_2025["key"], df_me2n_2022_2025[source_col]))
    df_me5a_2022_2025[target_col] = df_me5a_2022_2025["key"].map(temp_dict)

# 4. Geçici key sütunlarını kaldır
df_me2n_2022_2025.drop(columns=["key"], inplace=True)
df_me5a_2022_2025.drop(columns=["key"], inplace=True)


In [ ]:
# 1. Yeni sütun sırasını belirle
df_me2n_2022_2025_analog_yeni_sira = [
    'ГОД', 'Документ закупки', 'Позиция', 'Группа закупок',
    'Закуп. организация','Создал', 'Индикатор удаления', 'Дата документа',
    'Наш знак', 'Ваш код', 'Имя поставщика', 'Заявка',  'Позиция заявки',
    'Материал_ME2N','Материал_ME5A','Наименование RU_ЗП','Наименование TR_ЗП',
    'RU Наименование_ВЗ','TR Наименование_ВЗ',
    'Объем заказа', 'еще поставить (количество)','Уже поставлено (количество)',
    'Складская ЕИ','Количество в СЕИ','Единица цены','Цена нетто',
    'СтоимЗаказа нетто','Еще для поставки (стоимость)','Валюта',
    'Вид докум. закупки','Код налога','Дата поставки','ЕИ заказа на постав.',
    'Инвентарный номер'
]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
df_me2n_2022_2025_analog =df_me2n_2022_2025_analog[df_me2n_2022_2025_analog_yeni_sira]


In [ ]:
# 1. Yeni sütun sırasını belirle
df_me2n_2022_2025_yeni_sira = [
      'ГОД', 'Документ закупки', 'Позиция','Создал', 'Группа закупок',
      'Закуп. организация', 'Индикатор удаления', 'Дата документа',
      'Наш знак', 'Ваш код', 'Имя поставщика', 'Заявка','Позиция заявки',
      'Материал','Инвентарный номер', 'Наименование RU','Наименование TR',
      'Объем заказа','Складская ЕИ','еще поставить (количество)',
      'Еще для поставки (стоимость)','Уже поставлено (количество)',
      'Для фактурирования (стоим.)','Для фактурирования (колич.)',
      'Количество в СЕИ', 'Единица цены','СтоимЗаказа нетто','Валюта',
      'Цена нетто','Вид докум. закупки', 'Код налога','Краткий текст',
      'Дата поставки', 'ЕИ заказа на постав.','ТипДокумЗакупки',
      'Тип неполноты данных','Группа материалов','Материал_ME2N', 'Материал_ME5A'

]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
df_me2n_2022_2025 =df_me2n_2022_2025[df_me2n_2022_2025_yeni_sira]

In [ ]:
df_me5a_2022_2025.rename(columns={"'Дата заявки": "Дата_заявки"}, inplace=True)


In [ ]:
df_me5a_2022_2025_yeni_sira = [
       'ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3','ЗатребованКолич',
       'Объем заказа', 'еще поставить (количество)','Еще для поставки (стоимость)',
       'Для фактурирования (колич.)','Для фактурирования (стоим.)', 'Имя поставщика',
       'ЗП_Создал','Наш знак', 'Цена нетто', 'Валюта', 'ЕИ заказа на постав.',
       'Единица цены', 'Заказ на поставку', 'Позиция ЗкзНаПостав','Индикатор удаления_зп',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения','Дата заказа', 'Долгосрочный договор',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование',  'Дополнительное','Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', 'Дата_заявки', 'B3-B3POZ','Материал_ME5A'

]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
df_me5a_2022_2025 =df_me5a_2022_2025[df_me5a_2022_2025_yeni_sira]

In [ ]:
df_calisilan_yeni_sira = [
    'ГОД','zp_zppoz','B3-B3POZ','Документ_закупки', 'Зп_Поз',
    'Индикатор удаления','неполн.', 'Материал', 'Заявка', 'Позиция заявки',
    'RU Наименование', 'TR Наименование',
    'Инвентарный номер в SAS', 'Количество ЗП',
    'Минимальная цена ЗП', 'Минимальная цена ЗП.1',
    'Минимальная цена ЗП.2','Единица ЗП', 'Цена нетто ЗП', 'Итого цена нетто',
    'Валюта ЗП','Вид докум. закупки', 'Дата создания', 'Дата поставки',
    'Дата доставки, одобренная Поставщиком', 'Группа закупок',
    'Название ГрЗакупок', 'БЕ', 'Название фирмы', 'Закуп. организация',
    'Название ЗакупОрг', 'Завод', 'Имя 1', 'Поставщик', 'Группа материалов',
    'Название группы', 'Краткий текст', 'Создал', 'Вид докум. закупки.1',
    'Обозначение ВидДокум', 'Дата создания.1', 'Создал.1','Группа закупок.1',
    'Название ГрЗакупок.1', 'ЕИ заказа на постав.','Единица цены',
    'Цена заказа нетто', 'Стоимость брутто', 'Валюта','Недопоставленное количество',
    'Количество поставки', 'Сумма брутто','Валюта.1', 'Количество по счету', 'Дата проводки',
    'Количество по счету.1', 'Валюта.2', 'Остаток количества по счету',
    'Остаток суммы по счету', 'Автор Внутреннего заказа', 'Количество по счету.2',
    'Значок "поставка завершена"','Дата контракта', 'Дата вступления в силу контракта',
    'Дата окончания действия договора', 'Склад', '№ комиссии','имя_поставщика'
]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
df_calisilan =df_calisilan[df_calisilan_yeni_sira]

In [ ]:
# 1. Yeni sütun sırasını belirle
zmb51_yeni_sira = [
    'ГОД', 'Вид движения', 'Партия', 'Материал', 'Заказ на поставку',
    'Позиция', 'Наш знак', 'Ваш код', 'Внутренний заказ', 'Позиция заявки',
    'Склад', 'Дата ввода', 'Время ввода', 'Дата документа', 'Дата проводки',
    'Документ материала', 'Поз. док. материала', 'Признак 2', 'Ссылка',
    'Краткий текст материала', 'Кол-во в ЕИ ввода', 'Объем заказа',
    'ЕИ ввода','Сумма во ВВ', 'Количество', 'Поставщик','Имя поставщика',
    'Накладная','Текст заголовка документа', 'Заявитель', 'Инвентарный номер',
    'Имя пользователя', 'БЕ', 'Базовая ЕИ', 'ГодДокумМатериала', 'Валюта',
    'Завод', 'Имя 1', 'Основное средство', 'ЕИ цены заказа','ЕИ заказа на постав.'

]

# 2. DataFrame'i yeni sütun sırasına göre düzenle
birlesik_df_zmb51 = birlesik_df_zmb51[zmb51_yeni_sira]

In [ ]:
#ME5A
df_me5a_2022_2025["ГОД"] = pd.to_numeric(df_me5a_2022_2025["ГОД"], errors="coerce").astype("Int64")
df_me5a_2022_2025["Заявка"] = pd.to_numeric(df_me5a_2022_2025["Заявка"], errors="coerce").astype("Int64")
df_me5a_2022_2025["Материал"] = pd.to_numeric(df_me5a_2022_2025["Материал"], errors="coerce").astype("Int64")
df_me5a_2022_2025["Заказ на поставку"] = pd.to_numeric(df_me5a_2022_2025["Заказ на поставку"], errors="coerce").astype("Int64")
df_me5a_2022_2025["Позиция ЗкзНаПостав"] = pd.to_numeric(df_me5a_2022_2025["Позиция ЗкзНаПостав"], errors="coerce").astype("Int64")
df_me5a_2022_2025["ЗатребованКолич"] = pd.to_numeric(df_me5a_2022_2025["ЗатребованКолич"], errors="coerce").astype("float64")
df_me5a_2022_2025["Заказанное к-во"] = pd.to_numeric(df_me5a_2022_2025["Заказанное к-во"], errors="coerce").astype("float64")
df_me5a_2022_2025["Объем заказа"] = pd.to_numeric(df_me5a_2022_2025["Объем заказа"], errors="coerce").astype("float64")
df_me5a_2022_2025["еще поставить (количество)"] = pd.to_numeric(df_me5a_2022_2025["еще поставить (количество)"], errors="coerce").astype("float64")
df_me5a_2022_2025["Еще для поставки (стоимость)"] = pd.to_numeric(df_me5a_2022_2025["Еще для поставки (стоимость)"], errors="coerce").astype("float64")
df_me5a_2022_2025["Для фактурирования (колич.)"] = pd.to_numeric(df_me5a_2022_2025["Для фактурирования (колич.)"], errors="coerce").astype("float64")
df_me5a_2022_2025["Для фактурирования (стоим.)"] = pd.to_numeric(df_me5a_2022_2025["Для фактурирования (стоим.)"], errors="coerce").astype("float64")
#ME2N
df_me2n_2022_2025["ГОД"] = pd.to_numeric(df_me2n_2022_2025["ГОД"], errors="coerce").astype("Int64")
df_me2n_2022_2025["Документ закупки"] = pd.to_numeric(df_me2n_2022_2025["Документ закупки"], errors="coerce").astype("Int64")
df_me2n_2022_2025["Позиция"] = pd.to_numeric(df_me2n_2022_2025["Позиция"], errors="coerce").astype("Int64")
df_me2n_2022_2025["Заявка"] = pd.to_numeric(df_me2n_2022_2025["Заявка"], errors="coerce").astype("Int64")
df_me2n_2022_2025["Материал"] = pd.to_numeric(df_me2n_2022_2025["Материал"], errors="coerce").astype("Int64")
df_me2n_2022_2025["Объем заказа"] = pd.to_numeric(df_me2n_2022_2025["Объем заказа"], errors="coerce").astype("float64")
df_me2n_2022_2025["еще поставить (количество)"] = pd.to_numeric(df_me2n_2022_2025["еще поставить (количество)"], errors="coerce").astype("float64")
df_me2n_2022_2025["Количество в СЕИ"] = pd.to_numeric(df_me2n_2022_2025["Количество в СЕИ"], errors="coerce").astype("float64")
df_me2n_2022_2025["Цена нетто"] = pd.to_numeric(df_me2n_2022_2025["Цена нетто"], errors="coerce").astype("float64")
df_me2n_2022_2025["СтоимЗаказа нетто"] = pd.to_numeric(df_me2n_2022_2025["СтоимЗаказа нетто"], errors="coerce").astype("float64")
df_me2n_2022_2025["Еще для поставки (стоимость)"] = pd.to_numeric(df_me2n_2022_2025["Еще для поставки (стоимость)"], errors="coerce").astype("float64")
#zmb51
birlesik_df_zmb51["ГОД"] = pd.to_numeric(birlesik_df_zmb51["ГОД"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Партия"] = pd.to_numeric(birlesik_df_zmb51["Партия"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Заказ на поставку"] = pd.to_numeric(birlesik_df_zmb51["Заказ на поставку"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Позиция"] = pd.to_numeric(birlesik_df_zmb51["Позиция"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Внутренний заказ"] = pd.to_numeric(birlesik_df_zmb51["Внутренний заказ"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Документ материала"] = pd.to_numeric(birlesik_df_zmb51["Документ материала"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Поз. док. материала"] = pd.to_numeric(birlesik_df_zmb51["Поз. док. материала"], errors="coerce").astype("Int64")
birlesik_df_zmb51["Кол-во в ЕИ ввода"] = pd.to_numeric(birlesik_df_zmb51["Кол-во в ЕИ ввода"], errors="coerce").astype("float64")
birlesik_df_zmb51["Сумма во ВВ"] = pd.to_numeric(birlesik_df_zmb51["Сумма во ВВ"], errors="coerce").astype("float64")
birlesik_df_zmb51["Количество"] = pd.to_numeric(birlesik_df_zmb51["Количество"], errors="coerce").astype("float64")
birlesik_df_zmb51["Поставщик"] = pd.to_numeric(birlesik_df_zmb51["Поставщик"], errors="coerce").astype("Int64")
#zmm059
df_calisilan["ГОД"] = pd.to_numeric(df_calisilan["ГОД"], errors="coerce").astype("Int64")
df_calisilan["Документ_закупки"] = pd.to_numeric(df_calisilan["Документ_закупки"], errors="coerce").astype("Int64")
df_calisilan["Зп_Поз"] = pd.to_numeric(df_calisilan["Зп_Поз"], errors="coerce").astype("Int64")
df_calisilan["Материал"] = pd.to_numeric(df_calisilan["Материал"], errors="coerce").astype("Int64")
df_calisilan["Заявка"] = pd.to_numeric(df_calisilan["Заявка"], errors="coerce").astype("Int64")
df_calisilan["Позиция заявки"] = pd.to_numeric(df_calisilan["Позиция заявки"], errors="coerce").astype("Int64")
df_calisilan["Количество ЗП"] = pd.to_numeric(df_calisilan["Количество ЗП"], errors="coerce").astype("float64")
df_calisilan["Минимальная цена ЗП"] = pd.to_numeric(df_calisilan["Минимальная цена ЗП"], errors="coerce").astype("float64")
df_calisilan["Минимальная цена ЗП.1"] = pd.to_numeric(df_calisilan["Минимальная цена ЗП.1"], errors="coerce").astype("float64")
df_calisilan["Единица ЗП"] = pd.to_numeric(df_calisilan["Единица ЗП"], errors="coerce").astype("Int64")
df_calisilan["Цена нетто ЗП"] = pd.to_numeric(df_calisilan["Цена нетто ЗП"], errors="coerce").astype("float64")
df_calisilan["Итого цена нетто"] = pd.to_numeric(df_calisilan["Итого цена нетто"], errors="coerce").astype("float64")
df_calisilan["Поставщик"] = pd.to_numeric(df_calisilan["Поставщик"], errors="coerce").astype("Int64")
#me2n_analog
df_me2n_2022_2025_analog["ГОД"] = pd.to_numeric(df_me2n_2022_2025["ГОД"], errors="coerce").astype("Int64")
df_me2n_2022_2025_analog["Документ закупки"] = pd.to_numeric(df_me2n_2022_2025["Документ закупки"], errors="coerce").astype("Int64")
df_me2n_2022_2025_analog["Заявка"] = pd.to_numeric(df_me2n_2022_2025["Заявка"], errors="coerce").astype("Int64")
df_me2n_2022_2025_analog["Материал"] = pd.to_numeric(df_me2n_2022_2025["Материал"], errors="coerce").astype("Int64")
df_me2n_2022_2025_analog["Объем заказа"] = pd.to_numeric(df_me2n_2022_2025["Объем заказа"], errors="coerce").astype("float64")
df_me2n_2022_2025_analog["еще поставить (количество)"] = pd.to_numeric(df_me2n_2022_2025["еще поставить (количество)"], errors="coerce").astype("float64")
df_me2n_2022_2025_analog["Количество в СЕИ"] = pd.to_numeric(df_me2n_2022_2025["Количество в СЕИ"], errors="coerce").astype("float64")
df_me2n_2022_2025_analog["Цена нетто"] = pd.to_numeric(df_me2n_2022_2025["Цена нетто"], errors="coerce").astype("float64")
df_me2n_2022_2025_analog["СтоимЗаказа нетто"] = pd.to_numeric(df_me2n_2022_2025["СтоимЗаказа нетто"], errors="coerce").astype("float64")
df_me2n_2022_2025_analog["Еще для поставки (стоимость)"] = pd.to_numeric(df_me2n_2022_2025["Еще для поставки (стоимость)"], errors="coerce").astype("float64")

#zmm067
df_zmm067_2022_2025.rename(columns={"SAT Yaratma Tarihi": "Дата Заявка"}, inplace=True)
df_zmm067_2022_2025.rename(columns={"Sipariş Silme Göstergesi": "Индикатор удаления заказа"}, inplace=True)
df_zmm067_2022_2025.rename(columns={"SAT Teslimat Tarihi": "Дата доставки заявка"}, inplace=True)
df_zmm067_2022_2025.rename(columns={"Talep Silme Göstergesi": "Индикатор удаления заявка"}, inplace=True)
df_zmm067_2022_2025["Объем заказа"] = pd.to_numeric(df_zmm067_2022_2025["Объем заказа"], errors="coerce").astype("float64")

In [ ]:
df_calisilan_cleaned =df_calisilan.copy()

In [ ]:
df_me5a_cleaned.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', ''Дата заявки'],
      dtype='object')

In [ ]:
df_me5a_cleaned = df_me5a_cleaned[~df_me5a_cleaned["Индикатор удаления_B3"]]

In [ ]:
# sadece NaN olanları tutar; boş string ('') varsa bu satırlar silinebilir
df_calisilan_cleaned = df_calisilan_cleaned[df_calisilan_cleaned['Индикатор удаления'].isna()]


In [ ]:
df_calisilan_cleaned['Индикатор удаления']

,Индикатор удаления
0,<NA>
1,<NA>
2,<NA>
3,<NA>
4,<NA>
...,...
751753,<NA>
751754,<NA>
751755,<NA>
751756,<NA>


In [ ]:
df_calisilan_cleaned.columns


Index(['ГОД', 'zp_zppoz', 'B3-B3POZ', 'Документ_закупки', 'Зп_Поз',
       'Индикатор удаления', 'неполн.', 'Материал', 'Заявка', 'Позиция заявки',
       'RU Наименование', 'TR Наименование', 'Инвентарный номер в SAS',
       'Количество ЗП', 'Минимальная цена ЗП', 'Минимальная цена ЗП.1',
       'Минимальная цена ЗП.2', 'Единица ЗП', 'Цена нетто ЗП',
       'Итого цена нетто', 'Валюта ЗП', 'Вид докум. закупки', 'Дата создания',
       'Дата поставки', 'Дата доставки, одобренная Поставщиком',
       'Группа закупок', 'Название ГрЗакупок', 'БЕ', 'Название фирмы',
       'Закуп. организация', 'Название ЗакупОрг', 'Завод', 'Имя 1',
       'Поставщик', 'Группа материалов', 'Название группы', 'Краткий текст',
       'Создал', 'Вид докум. закупки.1', 'Обозначение ВидДокум',
       'Дата создания.1', 'Создал.1', 'Группа закупок.1',
       'Название ГрЗакупок.1', 'ЕИ заказа на постав.', 'Единица цены',
       'Цена заказа нетто', 'Стоимость брутто', 'Валюта',
       'Недопоставленное количест

In [ ]:
df_me5a_cleaned.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', ''Дата заявки'],
      dtype='object')

In [ ]:
df_me5a_cleaned.loc[:, 'zp_zppoz'] = (
    df_me5a_cleaned['Заказ на поставку'].astype(str) + '_' +
    df_me5a_cleaned['Позиция ЗкзНаПостав'].astype(str)
)

In [ ]:
df_me5a_cleaned.loc[:, 'B3-POZ'] = (
    df_me5a_cleaned['Заявка'].astype(str) + '_' +
    df_me5a_cleaned['Позиция заявки'].astype(str)
)

In [ ]:
df_me5a_cleaned.loc[df_me5a_cleaned['Заказ на поставку'].notna(), 'Заказ на поставку'] = pd.NA
df_me5a_cleaned.loc[df_me5a_cleaned['Позиция ЗкзНаПостав'].notna(), 'Позиция ЗкзНаПостав'] = pd.NA

In [ ]:
df_me5a_cleaned['ЗатребованКолич']

,ЗатребованКолич
0,2
1,870
2,870
4,870
5,4
...,...
379995,8
379996,30
379997,30
379998,200


In [ ]:
df_me5a_cleaned.loc[:, "Заказанное к-во"] = pd.to_numeric(
    df_me5a_cleaned["Заказанное к-во"], errors="coerce"
).astype("float64")

In [ ]:
df_me5a_cleaned.loc[:, "ЗатребованКолич"] = pd.to_numeric(
    df_me5a_cleaned["ЗатребованКолич"], errors="coerce"
).astype("float64")

In [ ]:
df_me5a_cleaned.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', ''Дата заявки', 'zp_zppoz', 'B3-POZ'],
      dtype='object')

In [ ]:
df_calisilan_cleaned.columns

Index(['ГОД', 'zp_zppoz', 'B3-B3POZ', 'Документ_закупки', 'Зп_Поз',
       'Индикатор удаления', 'неполн.', 'Материал', 'Заявка', 'Позиция заявки',
       'RU Наименование', 'TR Наименование', 'Инвентарный номер в SAS',
       'Количество ЗП', 'Минимальная цена ЗП', 'Минимальная цена ЗП.1',
       'Минимальная цена ЗП.2', 'Единица ЗП', 'Цена нетто ЗП',
       'Итого цена нетто', 'Валюта ЗП', 'Вид докум. закупки', 'Дата создания',
       'Дата поставки', 'Дата доставки, одобренная Поставщиком',
       'Группа закупок', 'Название ГрЗакупок', 'БЕ', 'Название фирмы',
       'Закуп. организация', 'Название ЗакупОрг', 'Завод', 'Имя 1',
       'Поставщик', 'Группа материалов', 'Название группы', 'Краткий текст',
       'Создал', 'Вид докум. закупки.1', 'Обозначение ВидДокум',
       'Дата создания.1', 'Создал.1', 'Группа закупок.1',
       'Название ГрЗакупок.1', 'ЕИ заказа на постав.', 'Единица цены',
       'Цена заказа нетто', 'Стоимость брутто', 'Валюта',
       'Недопоставленное количест

In [ ]:
# 1️⃣ Önce df_calisilan_cleaned'deki anahtarları benzersiz hale getiriyoruz
df_calisilan_unique = df_calisilan_cleaned.drop_duplicates(subset='B3-B3POZ')

# 2️⃣ Merge işlemini yapıyoruz (her iki sütunu da ekliyoruz)
df_me5a_cleaned = df_me5a_cleaned.merge(
    df_calisilan_unique[['B3-B3POZ', 'Документ_закупки', 'Зп_Поз']],
    left_on='B3-POZ',
    right_on='B3-B3POZ',
    how='left'
)

# 3️⃣ Dolularını kendi sütunlarına yazıyoruz
df_me5a_cleaned['Заказ на поставку'] = df_me5a_cleaned['Документ_закупки'].combine_first(
    df_me5a_cleaned['Заказ на поставку']
)
df_me5a_cleaned['Позиция ЗкзНаПостав'] = df_me5a_cleaned['Зп_Поз'].combine_first(
    df_me5a_cleaned['Позиция ЗкзНаПостав']
)

# 4️⃣ Geçici sütunları güvenli şekilde sil
cols_to_drop = [col for col in ['Документ_закупки', 'Зп_Поз', 'B3-B3POZ'] if col in df_me5a_cleaned.columns]
df_me5a_cleaned.drop(columns=cols_to_drop, inplace=True)


In [ ]:
df_me5a_cleaned.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', ''Дата заявки', 'zp_zppoz', 'B3-POZ'],
      dtype='object')

In [ ]:
df_me5a_cleaned['silme_durumu'] = np.where(
    df_me5a_cleaned['Заказанное к-во'].isna(),  # Eğer boşsa
    'dursun',
    np.where(
        df_me5a_cleaned['Заказанное к-во'] < df_me5a_cleaned['ЗатребованКолич'],
        'dursun',
        'sil'
    )
)

In [ ]:
print(df_me5a_cleaned['silme_durumu'].value_counts())

silme_durumu
sil       260755
dursun     90657
Name: count, dtype: int64


In [ ]:
df_me5a_cleaned.columns

Index(['ГОД', 'Заявка', 'Позиция заявки', 'Материал', 'Индикатор удаления_B3',
       'Затребовал', 'Индик. выдачи', 'Группа деблокир.',
       'Статус обработки заявки', 'Дата заявки', 'Группа закупок',
       'Дата изменения', 'Заказ на поставку', 'Позиция ЗкзНаПостав',
       'Дата заказа', 'Долгосрочный договор', 'ЗатребованКолич',
       'Заказанное к-во', 'Единица измерения', 'RU Наименование',
       'TR Наименование', 'Инвентарный номер', 'Краткий текст',
       'Закуп. организация', 'Создал', 'ПоступлМатериала', 'Поступление счета',
       'Номер проекта', 'Группа материалов', 'Дата поставки',
       'Завод-поставщик', 'Вид документа', 'Статус обработки',
       'Индикатор создания', 'Стратегия деблокир.', 'НДП-материал',
       'Номер потребности', 'Объем дефицита', 'Тип даты поставки',
       'Дата деблокирования', ''Дата заявки', 'zp_zppoz', 'B3-POZ',
       'silme_durumu'],
      dtype='object')

In [ ]:
df_me5a_cleaned = df_me5a_cleaned.drop(
    df_me5a_cleaned[
        df_me5a_cleaned['Заказ на поставку'].notna() &
        (df_me5a_cleaned['silme_durumu'] == 'sil')
    ].index
)

In [ ]:
print(df_me5a_cleaned['Заказ на поставку'].value_counts())

Заказ на поставку
3100010337    260
3100004384    119
3100004876    104
3100011876     77
3100011264     54
             ... 
3100002674      1
3100000695      1
3100012591      1
3100005235      1
3100000164      1
Name: count, Length: 1046, dtype: int64


In [ ]:
for col in ["Заказ на поставку", "Позиция ЗкзНаПостав"]:
    # Boş değerleri koru, dolu olanları integer yap
    df_me5a_cleaned[col] = df_me5a_cleaned[col].apply(
        lambda x: pd.NA if pd.isna(x) else int(x) if float(x).is_integer() else float(x)
    ).astype("Int64")  # nullable integer dtype

In [ ]:
df_me5a_cleaned[['Заявка','Позиция заявки','Заказ на поставку','Заказанное к-во','ЗатребованКолич']].sample(n=5)

,Заявка,Позиция заявки,Заказ на поставку,Заказанное к-во,ЗатребованКолич
208671,2200012745,50,<NA>,0.0,10.0
198086,9900002686,1360,<NA>,0.0,4.0
211117,2200012932,170,<NA>,0.0,53.0
345463,2200000547,480,<NA>,0.0,20.0
117798,2200008677,280,<NA>,0.0,1.0


In [ ]:
# 1. Zorunlu temizleme işlemleri (orijinal sütunların üzerine yazar)
df_me5a_cleaned["Заявка"] = df_me5a_cleaned["Заявка"].apply(temizle_ve_cevir)
df_me5a_cleaned["Позиция заявки"] = df_me5a_cleaned["Позиция заявки"].apply(temizle_ve_cevir)

df_zmm067_2022_2025["Заявка"] = df_zmm067_2022_2025["Заявка"].apply(temizle_ve_cevir)
df_zmm067_2022_2025["Позиция заявки"] = df_zmm067_2022_2025["Позиция заявки"].apply(temizle_ve_cevir)

# 2. Anahtar oluştur
df_me5a_cleaned["zayavka_poz"] = df_me5a_cleaned["Заявка"] + "_" + df_me5a_cleaned["Позиция заявки"]
df_zmm067_2022_2025["zayavka_poz"] = df_zmm067_2022_2025["Заявка"] + "_" + df_zmm067_2022_2025["Позиция заявки"]

# 3. Dict oluştur
dict_dop = dict(zip(df_zmm067_2022_2025["zayavka_poz"], df_zmm067_2022_2025["Дополнительное"]))

# 4. map ile yeni sütun ekle
df_me5a_cleaned["Дополнительное"] = df_me5a_cleaned["zayavka_poz"].map(dict_dop)

#yeni oluşturulan sutunlara ihtiyaç kalmadı

df_me5a_cleaned = df_me5a_cleaned.drop(columns=["zayavka_poz"])
df_zmm067_2022_2025 = df_zmm067_2022_2025.drop(columns=["zayavka_poz"])

In [ ]:
df_me5a_cleaned.to_excel("df_me5a_cleaned.xlsx", index=False)
files.download("df_me5a_cleaned.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_zmm067_2022_2025.to_excel("zmm067_2022_2025.xlsx", index=False)
files.download("zmm067_2022_2025.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_me2n_2022_2025_analog.to_excel("me2n_2022_2025_analog_pos.xlsx", index=False)
files.download("me2n_2022_2025_analog_pos.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_me5a_2022_2025.to_excel("me5a_2022_2025.xlsx", index=False)
files.download("me5a_2022_2025.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_me2n_2022_2025.to_excel("me2n_2022_2025.xlsx", index=False)
files.download("me2n_2022_2025.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
birlesik_df_zmb51.to_excel("zmb51_2022_2025.xlsx", index=False)
files.download("zmb51_2022_2025.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_calisilan.to_excel("zmm059_2022_2025.xlsx", index=False, engine="xlsxwriter")
files.download("zmm059_2022_2025.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>